In [7]:
# include -
include("./src/solution.jl");

In [8]:
# load my state table -
path_to_state_table_file = "./src/data/State.csv"
stateTable = load_state_table(path_to_state_table_file)

,State,T,P,V
,Int64,Int64,Float64,Float64
1,1,300,3.0,8.314
2,2,300,0.75,33.256
3,3,100,0.25,33.256
4,4,100,1.0,8.314


In [3]:
# load some constants -
path_to_constants_file = "./src/data/Constants.json"
cd = load_constants_dictionary(path_to_constants_file)

Dict{String,Any} with 4 entries:
  "Rhat" => 0.8314
  "Cv"   => 20.686
  "R"    => 8.314
  "Cp"   => 29.0

In [9]:
# compute the path table -
pathTable = compute_cycle_values(stateTable,cd)

,path,Q,W,delta_U,delta_H,delta_S
,String,Float64,Float64,Float64,Float64,Float64
1,1 -> 2,345.77,-345.77,0.0,0.0,1.15257
2,2 -> 3,-4137.2,0.0,-4137.2,-5800.0,-22.7259
3,3 -> 4,-115.257,115.257,0.0,0.0,-1.15257
4,4 -> 1,4137.2,0.0,4137.2,5800.0,22.7259
5,cycle,230.513,-230.513,0.0,0.0,3.55271e-15


In [13]:
# compute the efficiency -
eff_stirling = compute_efficiency(stateTable, cd)

0.6666666666666666

In [14]:
# compute Carnot eff -

In [15]:
eff_carnot = 1 - (minimum(stateTable[!,:T])/maximum(stateTable[!,:T]))

0.6666666666666667

#### Solution strategy:
Start with the state table, find all values that you don't need to compute (because of the cycle, and what was given in the problem). This will leave estimating missing values using the IGL equation of state. When the state table is done, fill in the path table. Again, start by filling in values that you don't need to do any computation for e.g., constant volume = no work. Next, start with $\Delta{U}$, $\Delta{H}$ and $\Delta{S}$. To compute $\Delta{U}$ on $\mathcal{P}_{ij}$:

\begin{equation}
\Delta{U}_{ij} = C_{V}\Delta{T}_{ji}
\end{equation} 

while for $\Delta{H}$ you can use:

\begin{equation}
\Delta{H}_{ij} = C_{P}\Delta{T}_{ji}
\end{equation}

To compute the entropy change, it depends upon the type of path. For isothermal paths we have:

\begin{equation}
\Delta{S}_{ij} = R\ln\left(V_{j}/V_{i}\right)
\end{equation} 

while for constant volume paths, we can use:

\begin{equation}
\Delta{S}_{ij} = C_{V}\ln\left(T_{j}/T_{i}\right)
\end{equation}

Lastly, you can compute the heat and work either using the heat capacity for heat, or by the work expression:

\begin{equation}
W_{ij} = -\int_{V_{i}}^{V_{j}}PdV
\end{equation}

where you substitute the IGL equation of state into the integral to arrive at:

\begin{equation}
W_{ij} = -RT\ln\left(V_{j}/V_{i}\right)
\end{equation} for the isothermal paths. Remember that $\Delta{U}$ = Q+W.